# Práctica 2: Enfermedad Cardiovascular

Esta es la segunda práctica de la asignatura de **Análisis de Datos** del grado de *Ingeniería Informática* de la *UC3M* durante el curso 2020/21.


Autores:  
***Alba Reinders Sánchez***, 100383444, gr.83  
***Alejandro Valverde Mahou***, 100383383, gr.83

![](./images/img.jpg)

El objetivo de esta práctica es predecir si un paciente tiene o no una enfermedad cardiovascular. Se trata de un *problema de clasificación* con salida es binaria. 

Este sistema se quiere usar como ayuda al diagnóstico médico, por lo que es muy importante que el modelo tenga capacidad explicativa y su interpretación debe poder determinar qué atributos son más relevante para el diagnóstico.

El conjunto de datos está compuesto por **70000** ejemplos (pacientes), cada uno con **11** atributos. Más la clase a predecir.

## Tarea 1: Entrenamiento y evaluación de un árbol de decisión

Dado que se requiere de una capacidad explicatica, se plantea usar un **árbol de decisión**, usando, en este primer acercamiento, todas las variables disponibles.

### Carga de los datos

Antes de poder entrenar el modelo, es necesario cargar los datos del fichero.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("cardio_train.csv", sep=';')
header = data.columns
display(data)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


### Crear el modelo

Una vez se tienen los datos, ya se puede crear el modelo.

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
dtc = DecisionTreeClassifier()

### Entrenar el modelo

In [5]:
X = data[header[:-1]]
y = data[header[-1]]
dtc.fit(X, y)

DecisionTreeClassifier()

### Evaluar el modelo

Para evaluar el modelo se va a usar la **matriz de confusión** y, usando los valores que proporcione, se van a calcular:

 * **Tasa de aciertos**
 * **Precisión**
 * **Sensibilidad**
 * **Tasa de falsos positivos**
 * **Especificidad**
 * **Puntuación F1**

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
y_pred = dtc.predict(X)

In [8]:
def confusionMatrix(y, y_pred):
    true_matrix = ['Real Enfermedad', 'Real No Enfermedad']
    pred_matrix = ['Predicho Enfermedad', 'Predicho No Enfermedad']

    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

    return pd.DataFrame([[tp, fp], [fn, tn]], columns=true_matrix, index=pred_matrix)

In [9]:
matrix = confusionMatrix(y, y_pred)
display(matrix)

,Real Enfermedad,Real No Enfermedad
Predicho Enfermedad,34979,0
Predicho No Enfermedad,0,35021


El árbol de decisión, dado que está evaluando sobre el conjunto de entrenamiento, no hay incoherencias en los datos (dos ejemplos idénticos con clases diferentes), y no se ha especificado una profundidad del árbol máxima, siempre va a ser capaz de realizar una predicción perfecta.

Esto no indica que este árbol haya generado un buen clasificador, porque no se ha evaluado con ejemplos desconocidos para el modelo.

Aún así, se va a proceder a evaluar este modelo con las métricas mencionadas anteriormente, para poder realizar una explicación en mayor profundidad de las mismas, aunque sus valores no aporten demasiada información.

#### Tasa de aciertos

También llamada *accuracy*, y mide el porcentaje de instancias **correctamente clasificadas**. Aporta una visión general de como de bueno es el modelo desde una perspectiva general.

$\displaystyle accuracy = \frac{TP + TN}{TP + TN + FP + FN}$

In [10]:
from sklearn.metrics import accuracy_score

In [11]:
def accuracy(y, y_pred):
    return accuracy_score(y, y_pred) * 100

In [12]:
acc = accuracy(y, y_pred)
print("Se clasifican correctamente un " + str(acc) + "% de las instancias")

Se clasifican correctamente un 100.0% de las instancias


Como se ha mencionado anteriormente, estas métricas van a sacar valores perfectos.

#### Precisión

Identifica que porcentaje de instancias clasificadas como positivas, lo **son realmente**. Es útil para conocer la precisión del modelo cuando asegura que una instancia es positiva.

$\displaystyle precision = \frac{TP}{TP + FP}$

In [13]:
from sklearn.metrics import precision_score

In [14]:
def precision(y, y_pred):
    return precision_score(y, y_pred) * 100

In [15]:
prec = precision(y, y_pred)
print("Se clasifican correctamente un " + str(prec) + "% de las instancias positivas")

Se clasifican correctamente un 100.0% de las instancias positivas


#### Sensibilidad

También denominado *recall* o *sensitivity*. Indica qué porcentaje de las instancias positivas son clasificadas **correctamente**. Es útil para conocer la precisión del modelo para determinar que una instancia positiva lo es, o se equivoca.

$\displaystyle recall = \frac{TP}{TP + FN}$

In [16]:
from sklearn.metrics import recall_score

In [17]:
def recall(y, y_pred):
    return recall_score(y, y_pred) * 100

In [18]:
rec = recall(y, y_pred)
print("El modelo acierta un " + str(rec) + "% de las instancias positivas")

El modelo acierta un 100.0% de las instancias positivas


#### Tasa de falsos positivos

Denominada por sus siglas en inglés *FPR*. Indica qué porcentaje de las instancias negativas son clasificadas **incorrectamente**. Es útil para conocer la precisión del modelo para determinar que una instancia negativa lo es, o se equivoca. 

$\displaystyle FPR = \frac{FP}{FP + TN}$

In [25]:
def FPR(y, ypred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    return (fp/(fp + tn)) * 100

In [26]:
fpr = FPR(y, y_pred)
print("El modelo falla un " + str(fpr) + "% de las instancias negativas")

El modelo falla un 0.0% de las instancias negativas


#### Especificidad

Similar al *recall*, pero con el caso de los negativos. Indica qué porcentaje de las instancias negativas son clasificadas **correctamente**. Es útil para conocer la precisión del modelo para determinar que una instancia negativa lo es, o se equivoca.

$\displaystyle specificity = \frac{TN}{TN + FP}$

In [27]:
def specificity(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    return (tn/(fp + tn)) * 100

In [28]:
spec = specificity(y, y_pred)
print("El modelo acierta un " + str(spec) + "% de las instancias negativas")

El modelo acierta un 100.0% de las instancias negativas


#### Puntuación F1

Indica un valor que agrega la precisión y *recall* mediante una media armónica. El mejor valor es 1 y el peor, 0

$\displaystyle F1 = 2 \frac{precision · recall}{precision + recall}$

In [29]:
from sklearn.metrics import f1_score

In [31]:
def F1(y, y_pred):
    return f1_score(y, y_pred)

In [33]:
f1 = F1(y, y_pred)
print("La puntuación F1 del modelo es: " + str(f1))

La puntuación F1 del modelo es: 1.0


Para resumir, todas las precisiones son:

In [34]:
def evaluation(y, y_pred):
    acc = accuracy(y, y_pred)
    prec = precision(y, y_pred)
    rec = recall(y, y_pred)
    fpr = FPR(y, y_pred)
    spec = specificity(y, y_pred)
    f1 = F1(y, y_pred)

    print("Se clasifican correctamente un " + str(acc) + "% de las instancias")
    print("Se clasifican correctamente un " + str(prec) + "% de las instancias positivas")
    print("El modelo acierta un " + str(rec) + "% de las instancias positivas")
    print("El modelo falla un " + str(fpr) + "% de las instancias negativas")
    print("El modelo acierta un " + str(spec) + "% de las instancias negativas")
    print("La puntuación F1 del modelo es: " + str(f1))
    
    return acc, prec, rec, fpr, spec, f1

In [36]:
ev = evaluation(y, y_pred)

Se clasifican correctamente un 100.0% de las instancias
Se clasifican correctamente un 100.0% de las instancias positivas
El modelo acierta un 100.0% de las instancias positivas
El modelo falla un 0.0% de las instancias negativas
El modelo acierta un 100.0% de las instancias negativas
La puntuación F1 del modelo es: 1.0
